In [1]:
import numpy as np

In [2]:
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
import torch
import random
from torchvision import models, transforms
from scipy.spatial import distance_matrix
from tqdm import tqdm
import os

import json
import datetime

import time

c:\Users\thors\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\io\image.py:14: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
import sys
root = '../../'
sys.path.append(root)
from HelpfulFunctions.batchCreation import createBatch
from HelpfulFunctions.metrics import meanAveragePrecision

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
print(device)

cpu


In [5]:
X_train = torch.tensor( np.load( root + "Features/train_features_vgg16_cifar10.npy" ) )
y_train = np.load( root + "Features/train_labels_vgg16_cifar10.npy" )

X_test = torch.tensor( np.load( root + "Features/test_features_vgg16_cifar10.npy" ) )
y_test = np.load( root + "Features/test_labels_vgg16_cifar10.npy" )

------

In [6]:
def getAPN(a, pos, dMatrix, margin):
    aP = [a]+pos
    neg_i = [i for i in range(len(dMatrix)) if i not in aP ]

    outPut = []

    for p in  pos:
        posDist = dMatrix[a][p]
        
        legal_i = [i for i in neg_i if (posDist < dMatrix[a][i]) and (dMatrix[a][i] <= posDist+margin)] # i = "global" index of the Hash
        
        legal_dist = [dMatrix[a][i] for i in legal_i]
        
        if legal_dist == []: continue
        max_dist = min(legal_dist)

        n = legal_i[ legal_dist.index(max_dist) ]
        outPut.append( (a, p, n) )

    return outPut

In [7]:
def customBatch(X_train, y_train, pos_sample_bal, batchSize, pos_label):    
    pos_i = [ i for i in range(len(X_train)) if y_train[i] == pos_label ]
    neg_i = list(set( range(len(X_train)) ) - set(pos_i))

    pos_sample_size = int(batchSize*pos_sample_bal)
    neg_sample_size = batchSize - pos_sample_size

    pos_i = random.sample( pos_i, pos_sample_size)
    neg_i = random.sample( neg_i, neg_sample_size)

    X_sample = torch.stack( [X_train[i] for i in pos_i+neg_i] )
    y_sample = [y_train[i] for i in pos_i+neg_i]
    return X_sample, y_sample

In [8]:
def tripletLoss(bits, margin, batchSize, pos_sample_bal):
    t_start = time.time()
    movingAvg_window = 2000
    noImprove_breakVal = 2000
    
    L = []
    mean_loss = 0

    model = nn.Sequential(  nn.Linear(4096,1024),
                            nn.ReLU(),
                            nn.Linear(1024, bits),
                            nn.Sigmoid()
                            )
    
    model = model.to(device)

    optimizer = optim.Adam( model.parameters() )
    criterion = nn.TripletMarginLoss(p=1, # Manhatten Distance
                                    margin=margin
                                    )

    loss_list = []
    lowest_loss = 10000
    no_improves = 0

    y_unique = list(set(y_train))
    for i in tqdm( range(20000) ):
        if batchSize != None:   xBatch, yBatch = customBatch( X_train, y_train, pos_sample_bal, batchSize, random.sample(y_unique,1)[0] )
        else:                   xBatch, yBatch = createBatch(X_train, y_train, batchSize)

        xBatch = xBatch.to(device)

        results = model(xBatch)
        results_np = results.cpu().detach().numpy()
        dMatrix = distance_matrix(results_np, results_np, p=1)

        APN_list = []

        for label in set(yBatch):
            pos_i_list = [j for j in range(len(yBatch)) if yBatch[j] == label]
            for anchor_i in pos_i_list:
                pos_i = [j for j in pos_i_list if j != anchor_i]
                APN_list += getAPN(anchor_i, pos_i, dMatrix, margin)

        if len(APN_list) > 0:
            a_list = []
            p_list = []
            n_list = []

            for apn in APN_list:
                a_list.append(results[ apn[0] ])
                p_list.append(results[ apn[1] ])
                n_list.append(results[ apn[2] ])
            
            # === Improve Model ===
            optimizer.zero_grad()
            loss = criterion( torch.stack(a_list).to(device), 
                              torch.stack(p_list).to(device), 
                              torch.stack(n_list).to(device) )
            loss.backward()
            optimizer.step()

            loss_list.append( float(loss) )
            L.append( float(loss) )
        
        if (i > movingAvg_window):
            while len(L) > movingAvg_window:
                L = L[1:]

            mean_loss = sum(L) / len(L)

            if mean_loss < lowest_loss:
                # print(f"\033[92m {'+'}\033[00m", float(loss))
                lowest_loss = mean_loss
                no_improves = 0
            else:
                # print(f"\033[91m {'-'}\033[00m", float(loss))
                no_improves += 1
            
            if no_improves > noImprove_breakVal: break

        if (i % 500) == 0: 
            #print(f"Make Bacthes + get results: {t2-t1}\n", f"Distance Matrix: {t3-t2}\n", f"MAke APN List: {t4-t3}\n", f"Back Propegate: {t5-t4}")
            
            print(i, mean_loss , no_improves)
            

    hash_train = (model(X_train.to(device)).cpu().detach().numpy() > 0.5).astype(int)
    hash_test = (model(X_test.to(device)).cpu().detach().numpy() > 0.5).astype(int)
    map = meanAveragePrecision( hash_test , hash_train, y_test, y_train)

    delta_t = time.time() - t_start

    return model, loss_list, map, i, delta_t

#bits = 32
#margin = random.randint(0,5) # int(bits / 10)
#batchSize = 100
#pos_sample_bal = 0.50

#model, loss_list, map, i = tripletLoss(bits, margin, batchSize, pos_sample_bal)

In [9]:
def hpo():
    bits = 32
    margin          = random.uniform(0,5)
    batchSize       = random.randint(20,120)
    pos_sample_bal  = random.uniform(0.05, 0.5)

    if margin == 0: margin = 1

    hp = {"bits":bits,"margin":margin, "batchSize": batchSize, "pos_sample_bal":pos_sample_bal}
    print(hp)

    model, loss_list, map, i, delta_t = tripletLoss(bits, margin, batchSize, pos_sample_bal)

    res = {}
    res["hp"] = hp
    res["loss_list"] = loss_list
    res["map"] = map
    res["i"] = i
    res["delta_t"] = delta_t


    now = str(datetime.datetime.now())
    now = now[:now.index(".")].replace(" ", "_").replace(":", "-")

    newpath = f"{root}Results/HPO/TripletLoss/{os.environ['COMPUTERNAME']}"
    if not os.path.exists(newpath):
        os.makedirs(newpath)

    filePath = f"{root}Results/HPO/TripletLoss/{os.environ['COMPUTERNAME']}/{now}.json"

    with open(filePath, "w") as fp:
        json.dump(res , fp)

In [10]:
while True:
    hpo()

{'bits': 32, 'margin': 2.358648427107244, 'batchSize': 30, 'pos_sample_bal': 0.4521339383877155}


  0%|          | 3/20000 [00:00<12:38, 26.37it/s]

0 0 0


  3%|▎         | 507/20000 [00:15<09:43, 33.41it/s]

500 0 0


...

3500 1.4593501976132393 1499


 20%|██        | 4002/20000 [03:11<12:43, 20.95it/s]

4000 1.4753395026922227 1999



 92%|█████████▏| 9182/10000 [01:24<00:07, 109.00it/s]


KeyboardInterrupt: 